In [12]:
import pandas as pd
import numpy as np
import pytypo
import re
import emoji

from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers

from transformers import AutoTokenizer,TFAutoModel,AutoModel


c:\Users\User\Anaconda3\envs\metopen\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df_train = pd.read_csv('datasetHoax.csv')
df_train.head()

,narasi,judul,label
0,caler tryout radio talk show recently shared t...,pemakaian masker menyebabkan penyakit legionaires,1
1,ythseluruh angota grup sesuai instruksi gubern...,instruksi gubernur jateng tentang penilangan b...,1
2,jokowi adalah presiden terbaik dalam sejarah b...,foto jim rohn jokowi adalah presiden terbaik d...,1
3,maf mas dan mbak ini bukan politik tapi kenyat...,ini bukan politik tapi kenyatan pak jokowi ber...,1
4,kadrun jika lihat foto ini panas dingin,foto kadrun jika lihat foto ini panas dingin,1


In [3]:
df_train.isna().sum()

narasi    0
judul     0
label     0
dtype: int64

There is no missing value, we can continue to the next step.

In [5]:
def clean(text) : 

  text_cleaning_re = "@\S+|https?:\S+|http?:\S|[#]+|[^A-Za-z0-9]+"
  text_cleaning_hash = "#[A-Za-z0-9]+" 
  text_cleaning_num = "(^|\W)\d+"

  text = re.sub(text_cleaning_hash, " ", text).strip()
  text = re.sub(text_cleaning_num, " ", text).strip()
  text = re.sub(text_cleaning_re, " ", text).strip()
  
  text = text.strip()

  out = []
  for word in text.split() :
    # try : 
    #   out.append(word.replace(word, slang[word]))
    # except Exception as e : 
    out.append(word)
      
  return pytypo.correct_sentence(" ".join(out).strip())

In [9]:
def freeEmoji(text):
    allchars = [str for str in text]
    emoji_list = [c for c in allchars if c in emoji.EMOJI_DATA]
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])
    return clean_text

In [10]:

df_train["narasi"] = df_train["narasi"].apply(clean)
df_train["judul"] = df_train["judul"].apply(clean)


df_train["narasi"] = df_train["narasi"].apply(freeEmoji)
df_train["judul"] = df_train["judul"].apply(freeEmoji)


In [11]:
df_train.head()

,narasi,judul,label
0,caler tryout radio talk show recently shared t...,pemakaian masker menyebabkan penyakit legionaires,1
1,ythseluruh angota grup sesuai instruksi gubern...,instruksi gubernur jateng tentang penilangan b...,1
2,jokowi adalah presiden terbaik dalam sejarah b...,foto jim rohn jokowi adalah presiden terbaik d...,1
3,maf mas dan mbak ini bukan politik tapi kenyat...,ini bukan politik tapi kenyatan pak jokowi ber...,1
4,kadrun jika lihat foto ini panas dingin,foto kadrun jika lihat foto ini panas dingin,1


In [13]:
tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p2")
IndoBert = TFAutoModel.from_pretrained("indobenchmark/indobert-base-p2")

c:\Users\User\Anaconda3\envs\metopen\lib\site-packages\huggingface_hub\file_download.py:129: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\User\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some layers from the model checkpoint at indobenchmark/indobert-base-p2 were not used when initializing TFBertModel: ['mlm___cls', 'nsp___